In [ ]:
import yfinance as yf
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import pandas as pd

def buy_sell_strategy(starting_amount=1000000, num_intervals=20):
    # 주식 데이터 불러오기 (예시로 삼성전자 주식 데이터를 사용합니다)
    ticker = yf.Ticker('005930.KS')  # 삼성전자 종목코드: 005930.KS
    data = ticker.history(period='1d', start='2019-01-01', end='2021-12-31')  # 기간 설정

    # 날짜와 종가 컬럼 추출
    dates = data.index
    closing_prices = data['Close']

    # 시작금액으로 초기 투자금 설정
    investment = starting_amount

    # 데이터를 담을 리스트 생성
    result_data = []

    for i in range(1, num_intervals):
        start = (i - 1) * len(dates) // num_intervals
        end = i * len(dates) // num_intervals

        interval_prices = closing_prices[start:end]

        # 데이터가 없는 경우 건너뜀
        if len(interval_prices) == 0:
            continue

        max_price = interval_prices.max()
        min_price = interval_prices.min()

        result_data.append({
            'Interval': i,
            'Max Price': max_price,
            'Min Price': min_price
        })

        # 각 구간에서 최고점에서 팔고 최저점에서 사는 전략 수행
        investment = investment * (max_price / min_price)

    # 데이터프레임으로 변환
    result_df = pd.DataFrame(result_data)

    # 데이터프레임 출력
    print(result_df)


    # 그래프 크기 설정
    plt.figure(figsize=(25, 10))  # 가로 20인치, 세로 6인치

    # 그래프 그리기
    plt.plot(dates, closing_prices, linewidth=3)
    plt.xlabel('Dates')
    plt.ylabel('Stock Price')
    plt.title('Stock Price Trend')
    plt.xticks(rotation=45)  # x축 날짜 라벨 회전

    # 구간 나누기
    interval_width = len(dates) // num_intervals  # 구간의 너비 계산

    for i in range(1, num_intervals):
        x = dates[i * interval_width]  # 구간의 x 좌표
        plt.axvline(x=x, color='red', linestyle='--')

    # 구간 내에서 최고점과 최저점 표시
    for i in range(num_intervals - 1):
        start = (i - 1) * len(dates) // num_intervals
        end = i * len(dates) // num_intervals

        interval_prices = closing_prices[start:end]

        # 데이터가 없는 경우 건너뜀
        if len(interval_prices) == 0:
            continue

        max_price = interval_prices.max()
        min_price = interval_prices.min()

        max_idx = np.where(interval_prices == max_price)[0][0] + start
        min_idx = np.where(interval_prices == min_price)[0][0] + start

        plt.scatter(dates[max_idx], max_price, color='green', marker='^', s=100)
        plt.scatter(dates[min_idx], min_price, color='red', marker='v', s=100)


    # 일주일 간격으로 가로 축 눈금 설정
    ax = plt.gca()
    ax.xaxis.set_major_locator(mdates.MonthLocator())
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))

    plt.grid(True)
    plt.show()

    return investment

starting_amount = 1000000
final_amount = buy_sell_strategy(starting_amount)
profit = final_amount - starting_amount

print(f"시작금액: {starting_amount}원")
print(f"최종금액: {final_amount}원")
print(f"수익금액: {profit}원")

In [ ]:
import yfinance as yf
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import pandas as pd

def buy_sell_strategy(starting_amount=1000000, interval='1M'):
    # 주식 데이터 불러오기 (예시로 삼성전자 주식 데이터를 사용합니다)
    ticker = yf.Ticker('005930.KS')  # 삼성전자 종목코드: 005930.KS
    data = ticker.history(period='1d', start='2015-01-01', end='2021-12-31')  # 기간 설정

    # 날짜와 종가 컬럼 추출
    dates = data.index
    closing_prices = data['Close']

    # 시작금액으로 초기 투자금 설정
    investment = starting_amount

    # 데이터를 원하는 간격으로 리샘플링
    resampled_data = data.resample(interval).last()

    # 그래프 크기 설정
    plt.figure(figsize=(25, 10))  # 가로 20인치, 세로 6인치

    # 그래프 그리기
    plt.plot(dates, closing_prices, linewidth=3)
    plt.xlabel('Dates')
    plt.ylabel('Stock Price')
    plt.title('Stock Price Trend')
    plt.xticks(rotation=45)  # x축 날짜 라벨 회전

    # 구간 나누기
    for date in resampled_data.index:
        plt.axvline(x=date, color='red', linestyle='--')

    # 각 구간의 최고점과 최저점 표시
    for i in range(1, len(resampled_data)):
        start_date = resampled_data.index[i - 1]
        end_date = resampled_data.index[i]

        interval_prices = closing_prices.loc[(dates >= start_date) & (dates <= end_date)]

        # 데이터가 없는 경우 건너뜀
        if len(interval_prices) == 0:
            continue

        max_price = interval_prices.max()
        min_price = interval_prices.min()

        max_idx = interval_prices.idxmax()
        min_idx = interval_prices.idxmin()

        # 각 구간의 최대값과 최소값을 표시
        plt.scatter(max_idx, max_price, color='green', marker='^', s=100)
        plt.scatter(min_idx, min_price, color='red', marker='v', s=100)

        result_df = pd.DataFrame({
            'Interval': [i],
            'Max Price': [max_price],
            'Min Price': [min_price]
        })

        # 각 구간에서 최고점에서 팔고 최저점에서 사는 전략 수행
        investment = investment * (max_price / min_price)

        print(result_df)

    # 일주일 간격으로 가로 축 눈금 설정
    ax = plt.gca()
    ax.xaxis.set_major_locator(mdates.MonthLocator())
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))

    plt.grid(True)
    plt.show()

    return investment

starting_amount = 1000000
final_amount = buy_sell_strategy(starting_amount)
profit = final_amount - starting_amount

print(f"시작금액: {starting_amount}원")
print(f"최종금액: {final_amount}원")
print(f"수익금액: {profit}원")

In [ ]:
import yfinance as yf
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import pandas as pd

class StockAnalyzer:
    def __init__(self, ticker, start_date, end_date):
        self.ticker = ticker
        self.start_date = start_date
        self.end_date = end_date
        self.stock_data = self.get_stock_data()

    def get_stock_data(self):
        ticker_data = yf.Ticker(self.ticker)
        stock_data = ticker_data.history(period='1d', start=self.start_date, end=self.end_date)
        return stock_data

    def resample_data(self, interval):
        resampled_data = self.stock_data.resample(interval).last()
        return resampled_data

    def plot_stock_trend(self, resampled_data):
        dates = self.stock_data.index
        closing_prices = self.stock_data['Close']

        plt.figure(figsize=(25, 10))
        plt.plot(dates, closing_prices, linewidth=3)
        plt.xlabel('Dates')
        plt.ylabel('Stock Price')
        plt.title('Stock Price Trend')
        plt.xticks(rotation=45)

        for date in resampled_data.index:
            plt.axvline(x=date, color='red', linestyle='--')

        plt.grid(True)
        plt.show()

    def calculate_investment(self, starting_amount, resampled_data):
        investment = starting_amount
        result_data = []

        for i in range(1, len(resampled_data)):
            start_date = resampled_data.index[i - 1]
            end_date = resampled_data.index[i]

            interval_prices = self.stock_data['Close'].loc[(self.stock_data.index >= start_date) & (self.stock_data.index <= end_date)]

            if len(interval_prices) == 0:
                continue

            max_price = interval_prices.max()
            min_price = interval_prices.min()

            result_data.append({
                'Interval': i,
                'Max Price': max_price,
                'Min Price': min_price
            })

            investment = investment * (max_price / min_price)

        result_df = pd.DataFrame(result_data)
        return result_df, investment

    def analyze_stock(self, starting_amount=1000000, interval='1M'):
        resampled_data = self.resample_data(interval)
        self.plot_stock_trend(resampled_data)

        result_df, final_amount = self.calculate_investment(starting_amount, resampled_data)

        profit = final_amount - starting_amount

        print(f"시작금액: {starting_amount}원")
        print(f"최종금액: {final_amount}원")
        print(f"수익금액: {profit}원")
        print(result_df)

def main():
    ticker = '005930.KS'  # 삼성전자 종목코드: 005930.KS
    start_date = '2019-01-01'
    end_date = '2021-12-31'
    stock_analyzer = StockAnalyzer(ticker, start_date, end_date)
    stock_analyzer.analyze_stock()

if __name__ == "__main__":
    main()

In [ ]:
import yfinance as yf
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import pandas as pd

def buy_sell_strategy(starting_amount=1000000, interval='1M'):
    # 주식 데이터 불러오기 (예시로 삼성전자 주식 데이터를 사용합니다)
    ticker = yf.Ticker('035720.KS')  # 삼성전자 종목코드: 005930.KS
    data = ticker.history(period='1d', start='2019-01-01', end='2021-12-31')  # 기간 설정

    # 날짜와 종가 컬럼 추출
    dates = data.index
    closing_prices = data['Close']

    # 시작금액으로 초기 투자금 설정
    investment = starting_amount

    # 데이터를 원하는 간격으로 리샘플링
    resampled_data = data.resample(interval).last()

    # 그래프 크기 설정
    plt.figure(figsize=(25, 10))  # 가로 20인치, 세로 6인치

    # 그래프 그리기
    plt.plot(dates, closing_prices, linewidth=3)
    plt.xlabel('Dates')
    plt.ylabel('Stock Price')
    plt.title('Stock Price Trend')
    plt.xticks(rotation=45)  # x축 날짜 라벨 회전

    # 구간 나누기
    for date in resampled_data.index:
        plt.axvline(x=date, color='red', linestyle='--')

    # 각 구간의 최고점과 최저점 표시
    for i in range(1, len(resampled_data)):
        start_date = resampled_data.index[i - 1]
        end_date = resampled_data.index[i]

        interval_prices = closing_prices.loc[(dates >= start_date) & (dates <= end_date)]

        # 데이터가 없는 경우 건너뜀
        if len(interval_prices) == 0:
            continue

        max_price = interval_prices.max()
        min_price = interval_prices.min()

        max_idx = interval_prices.idxmax()
        min_idx = interval_prices.idxmin()

        # 각 구간의 최대값과 최소값을 표시
        plt.scatter(max_idx, max_price, color='green', marker='^', s=100)
        plt.scatter(min_idx, min_price, color='red', marker='v', s=100)

        result_df = pd.DataFrame({
            'Interval': [i],
            'Max Price': [max_price],
            'Min Price': [min_price]
        })

        # 각 구간에서 최고점에서 팔고 최저점에서 사는 전략 수행
        investment = investment * (max_price / min_price)

        print(result_df)

    # 일주일 간격으로 가로 축 눈금 설정
    ax = plt.gca()
    ax.xaxis.set_major_locator(mdates.MonthLocator())
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))

    plt.grid(True)
    plt.show()

    return investment

starting_amount = 1000000
final_amount = buy_sell_strategy(starting_amount)
profit = final_amount - starting_amount

print(f"시작금액: {starting_amount}원")
print(f"최종금액: {final_amount}원")
print(f"수익금액: {profit}원")